# IBM WatsonX + Exa Integration

This notebook demonstrates how to combine IBM WatsonX AI with Exa's web search capabilities to create an AI assistant that can search and use current information.

## 1. Setup

First, let's install the required packages:

In [ ]:
!pip install ibm-watsonx-ai exa-py python-dotenv

## 2. Environment Setup

Create a `.env` file with your API keys or set them directly here:

In [ ]:
import os
from dotenv import load_dotenv

# Load from .env file
load_dotenv()

# Or set directly (replace with your keys)
os.environ['IBM_WATSONX_API_KEY'] = 'your_ibm_key'
os.environ['IBM_WATSONX_URL'] = 'https://us-south.ml.cloud.ibm.com'
os.environ['IBM_WATSONX_PROJECT_ID'] = 'your_project_id'
os.environ['EXA_API_KEY'] = 'your_exa_key'

## 3. IBM WatsonX Client

Set up the IBM WatsonX client:

In [ ]:
from ibm_watsonx_ai import APIClient, Credentials

def get_client():
    """Create and return a configured IBM WatsonX AI client."""
    credentials = Credentials(
        url=os.getenv('IBM_WATSONX_URL'),
        api_key=os.getenv('IBM_WATSONX_API_KEY')
    )
    return APIClient(
        credentials=credentials,
        project_id=os.getenv('IBM_WATSONX_PROJECT_ID')
    )

## 4. Exa Search Client

Set up the Exa web search client:

In [ ]:
from typing import List, Dict, Any
from exa_py import Exa

class ExaSearchClient:
    def __init__(self, api_key: str):
        self.client = Exa(api_key=api_key)
    
    def search(self, query: str, num_results: int = 3) -> List[Dict[str, Any]]:
        response = self.client.search_and_contents(
            query,
            type="auto",
            num_results=num_results,
            text=True
        )
        
        formatted_results = []
        if isinstance(response, dict) and 'data' in response:
            for result in response['data'].get('results', []):
                if 'text' in result:
                    formatted_results.append({
                        "title": result.get('title', 'No title'),
                        "text": result['text'][:500] + "..." if len(result['text']) > 500 else result['text'],
                        "url": result.get('url', ''),
                        "score": result.get('score', 0.0)
                    })
        
        return formatted_results

## 5. Tool Definition

Define the web search tool:

In [ ]:
import json

def create_web_search_tool():
    return {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for current information. Always cite sources.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"]
            }
        }
    }

class ToolExecutor:
    def __init__(self, exa_client):
        self.exa_client = exa_client
    
    def execute_tool(self, tool_call):
        if tool_call["type"] != "function":
            return "Unsupported tool type"
        
        function_call = tool_call["function"]
        if function_call["name"] == "web_search":
            query = json.loads(function_call["arguments"])["query"]
            results = self.exa_client.search(query)
            
            return "\n\n".join(
                f"Source: {r['title']}\nURL: {r['url']}\nContent: {r['text']}\n"
                for r in results
            )

## 6. Main Model

Create the main model with tool calling:

In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference

class WatsonXModel:
    def __init__(self, exa_api_key):
        self.client_wrapper = get_client()
        self.exa_client = ExaSearchClient(exa_api_key)
        self.tool_executor = ToolExecutor(self.exa_client)
        
        self.model = ModelInference(
            model_id="mistralai/mistral-large",
            credentials=self.client_wrapper.credentials,
            project_id=self.client_wrapper.project_id
        )
    
    def ask(self, question: str) -> str:
        messages = [{"role": "user", "content": question}]
        tools = [create_web_search_tool()]
        
        # Get initial response with tool calls
        response = self.model.chat(messages=messages, tools=tools)
        
        # Handle tool calls if any
        if "choices" in response and response["choices"]:
            choice = response["choices"][0]
            if "message" in choice and "tool_calls" in choice["message"]:
                tool_calls = choice["message"]["tool_calls"]
                
                # Execute tools and get results
                tool_results = []
                for tool_call in tool_calls:
                    result = self.tool_executor.execute_tool(tool_call)
                    tool_results.append(result)
                
                # Add results to conversation
                messages.extend([
                    {
                        "role": "assistant",
                        "content": None,
                        "tool_calls": tool_calls
                    },
                    {
                        "role": "tool",
                        "content": "\n\n".join(tool_results),
                        "tool_call_id": tool_calls[0]["id"]
                    }
                ])
                
                # Get final response
                final_response = self.model.chat(messages=messages)
                if "choices" in final_response and final_response["choices"]:
                    return final_response["choices"][0]["message"]["content"].strip()
        
        return "No response generated"

## 7. Try It Out!

Let's test the AI with some questions:

In [ ]:
# Initialize the model
model = WatsonXModel(exa_api_key=os.getenv('EXA_API_KEY'))

# Try some questions
questions = [
    "What are the latest developments in AI regulation in 2024?",
    "What are recent breakthroughs in quantum computing?",
    "What are the current trends in renewable energy?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    print("\nAnswer:")
    print(model.ask(question))
    print("\n" + "-"*80)